# Planificación
#### Capítulos 10-11
----

Este cuaderno sirve como material de apoyo para los temas tratados en el **Capítulo 10 - Planificación clásica** y el **Capítulo 11 - Planificación y actuación en el mundo real** del libro *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*.
Este cuaderno utiliza implementaciones del módulo [planning.py](https://github.com/aimacode/aima-python/blob/master/planning.py).
Consulte [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) para obtener instrucciones.

Comenzaremos analizando los tipos de datos `PlanningProblem` y `Action` para definir problemas y acciones.
Luego, veremos cómo usarlos intentando planificar un viaje desde *Sibiu* a *Bucarest* a través del conocido mapa de Rumania, desde [search.ipynb](https://github.com/aimacode/aima-python/blob/master/search.ipynb)
seguido de algunos problemas de planificación comunes y métodos para resolverlos.

Comencemos importando todo desde el módulo de planificación.

In [79]:
from planning import *
from notebook import psource

## CONTENIDO

**Planificación clásica**
- Problema de planificación
- Acción
- Problemas de planificación
* Air cargo problem
* Problema con la rueda de repuesto
*Problema de torre de tres cuadras
* Problema de compras
*Problema de calcetines y zapatos.
* problema de la torta
- Resolución de problemas de planificación
* Plan gráfico
* Linealizar
* Planificador de pedidos parciales
<br>

**Planificación en el mundo real**
- Problema
-HLA
- Problemas de planificación
* Problema del taller de trabajo
* Doble problema de tenis
- Resolución de problemas de planificación
* Búsqueda Jerárquica
* Búsqueda Angelical

## Problema de planificación

PDDL significa Lenguaje de definición de dominio de planificación.
La clase `PlanningProblem` se utiliza para representar problemas de planificación en este módulo. Los siguientes atributos son esenciales para poder definir un problema:
* un estado inicial
* un conjunto de objetivos
* un conjunto de acciones viables que se pueden ejecutar en el espacio de búsqueda del problema

Vea el código fuente para ver cómo el código Python intenta realizarlos.

In [80]:
psource(PlanningProblem)

El atributo `init` es una expresión que forma la base de conocimiento inicial del problema.
<br>
El atributo `metas` es una expresión que indica los objetivos que debe alcanzar el problema.
<br>
Por último, "acciones" contiene una lista de objetos "Acción" que pueden ejecutarse en el espacio de búsqueda del problema.
<br>
El método `goal_test` comprueba si se ha alcanzado el objetivo.
<br>
El método "act" representa la acción dada y actualiza el estado actual.
<br>


## ACCIÓN

Para poder modelar adecuadamente un problema de planificación, es fundamental poder representar una Acción. Cada acción que modelamos requiere al menos tres cosas:
*condiciones previas que debe cumplir la acción
* los efectos de ejecutar la acción
* alguna expresión que represente la acción

El módulo modela acciones usando la clase `Action`

In [81]:
psource(Action)

Esta clase representa una acción dada la expresión, las condiciones previas y sus efectos.
Una lista "precond" almacena las condiciones previas de la acción y una lista "effect" almacena sus efectos.
Las condiciones previas y los efectos negativos se ingresan utilizando un símbolo "~" antes de la cláusula, que internamente tiene el prefijo "No" para que sea más fácil trabajar con ellos.
Por ejemplo, la negación de `At(obj, loc)` se ingresará como `~At(obj, loc)` y se representará internamente como `NotAt(obj, loc)`.
De manera equivalente, esto crea una nueva cláusula para cada literal negativo, eliminando la molestia de mantener dos bases de conocimiento separadas.
Esto simplifica enormemente algoritmos como "GraphPlan", como veremos más adelante.
El método `convert` toma una cadena de entrada, la analiza, elimina las conjunciones, si las hay, y devuelve una lista de objetos `Expr`.
El método `check_precond` comprueba si las condiciones previas para esa acción son válidas, dado un `kb`.
El método "act" lleva a cabo la acción sobre la base de conocimientos dada.

Ahora intentemos definir un problema de planificación utilizando estas herramientas. Como ya conocemos el mapa de Rumania, veamos si podemos planificar un viaje a través de un mapa simplificado de Rumania.

Aquí está nuestra definición de mapa simplificada:

In [82]:
from utils import *
# this imports the required expr so we can create our knowledge base

knowledge_base = [
    expr("Connected(Bucharest,Pitesti)"),
    expr("Connected(Pitesti,Rimnicu)"),
    expr("Connected(Rimnicu,Sibiu)"),
    expr("Connected(Sibiu,Fagaras)"),
    expr("Connected(Fagaras,Bucharest)"),
    expr("Connected(Pitesti,Craiova)"),
    expr("Connected(Craiova,Rimnicu)")
    ]

Agreguemos algunas proposiciones lógicas para completar nuestro conocimiento sobre cómo viajar por el mapa. Estas son las propiedades típicas de simetría y transitividad de las conexiones en un mapa. Ahora podemos estar seguros de que nuestra `base_de_conocimientos` comprende lo que realmente significa que dos ubicaciones estén conectadas en el sentido que los humanos suelen dar cuando usamos el término.

Agreguemos también nuestra ubicación inicial: *Sibiu* al mapa.

In [83]:
knowledge_base.extend([
     expr("Connected(x,y) ==> Connected(y,x)"),
     expr("Connected(x,y) & Connected(y,z) ==> Connected(x,z)"),
     expr("At(Sibiu)")
    ])

Ahora tenemos una base de conocimientos completa, que se puede ver así:

In [84]:
knowledge_base

[Connected(Bucharest, Pitesti),
 Connected(Pitesti, Rimnicu),
 Connected(Rimnicu, Sibiu),
 Connected(Sibiu, Fagaras),
 Connected(Fagaras, Bucharest),
 Connected(Pitesti, Craiova),
 Connected(Craiova, Rimnicu),
 (Connected(x, y) ==> Connected(y, x)),
 ((Connected(x, y) & Connected(y, z)) ==> Connected(x, z)),
 At(Sibiu)]

Ahora definimos posibles acciones para nuestro problema. Sabemos que podemos conducir entre cualquier lugar conectado. Pero, como se desprende de la lista [this](https://en.wikipedia.org/wiki/List_of_airports_in_Romania) de aeropuertos rumanos, también podemos volar directamente entre Sibiu, Bucarest y Craiova.

Podemos definir estas acciones de vuelo así:

In [85]:
#Sibiu to Bucharest
precond = 'At(Sibiu)'
effect = 'At(Bucharest) & ~At(Sibiu)'
fly_s_b = Action('Fly(Sibiu, Bucharest)', precond, effect)

#Bucharest to Sibiu
precond = 'At(Bucharest)'
effect = 'At(Sibiu) & ~At(Bucharest)'
fly_b_s = Action('Fly(Bucharest, Sibiu)', precond, effect)

#Sibiu to Craiova
precond = 'At(Sibiu)'
effect = 'At(Craiova) & ~At(Sibiu)'
fly_s_c = Action('Fly(Sibiu, Craiova)', precond, effect)

#Craiova to Sibiu
precond = 'At(Craiova)'
effect = 'At(Sibiu) & ~At(Craiova)'
fly_c_s = Action('Fly(Craiova, Sibiu)', precond, effect)

#Bucharest to Craiova
precond = 'At(Bucharest)'
effect = 'At(Craiova) & ~At(Bucharest)'
fly_b_c = Action('Fly(Bucharest, Craiova)', precond, effect)

#Craiova to Bucharest
precond = 'At(Craiova)'
effect = 'At(Bucharest) & ~At(Craiova)'
fly_c_b = Action('Fly(Craiova, Bucharest)', precond, effect)

Y el impulso actúa como este.

In [86]:
#Drive
precond = 'At(x)'
effect = 'At(y) & ~At(x)'
drive = Action('Drive(x, y)', precond, effect)

Nuestro objetivo se define como

In [87]:
goals = 'At(Bucharest)'

Finalmente, podemos definir una función que nos dirá cuando hemos llegado a nuestro destino, Bucarest.

In [88]:
def goal_test(kb):
    return kb.ask(expr('At(Bucharest)'))

Así, con todos los componentes establecidos, podemos definir el problema de planificación.

In [89]:
prob = PlanningProblem(knowledge_base, goals, [fly_s_b, fly_b_s, fly_s_c, fly_c_s, fly_b_c, fly_c_b, drive])

## PROBLEMAS DE PLANIFICACIÓN
---

## Air Cargo Problem

En el problema de la carga aérea, comenzamos con carga en dos aeropuertos, SFO y JFK. Nuestro objetivo es enviar cada carga al otro aeropuerto. Contamos con dos aviones para ayudarnos a realizar la tarea.
El problema se puede definir con tres acciones: Cargar, Descargar y Volar.
Veamos cómo se ha definido el problema `air_cargo` en el módulo.

In [90]:
psource(air_cargo)

**En(c, a):** La carga **'c'** está en el aeropuerto **'a'**.

**~En(c, a):** La carga **'c'** _no_ está en el aeropuerto **'a'**.

**In(c, p):** La carga **'c'** está en el avión **'p'**.

**~In(c, p):** La carga **'c'** _no_ está en el avión **'p'**.

**Cargo(c):** Declare **'c'** as cargo.

**Plano(p):** Declarar **'p'** como plano.

**Aeropuerto(a):** Declarar **'a'** como aeropuerto.



En el `estado_inicial`, tenemos la carga C1, avión P1 en el aeropuerto SFO y la carga C2, avión P2 en el aeropuerto JFK.
Nuestro objetivo es tener la carga C1 en el aeropuerto JFK y la carga C2 en el aeropuerto SFO. Discutiremos cómo lograrlo. Definamos ahora un objeto del problema `air_cargo`:

In [91]:
airCargo = air_cargo()

Antes de realizar cualquier acción, comprobaremos si `airCargo` ha alcanzado su objetivo:

In [92]:
print(airCargo.goal_test())

False


Devuelve False porque aún no se ha alcanzado el estado objetivo. Ahora, definimos la secuencia de acciones que se deben tomar para lograr el objetivo.
Luego se realizan las acciones sobre el Problema de Planificación de la “Carga Aérea”.

Las acciones que tenemos a nuestra disposición son las siguientes: Cargar, Descargar, Volar

**Carga(c, p, a):** Cargue la carga **'c'** en el avión **'p'** desde el aeropuerto **'a'**.

**Fly(p, f, t):** Vuela el avión **'p'** desde el aeropuerto **'f'** al aeropuerto **'t'**.

**Descargar(c, p, a):** Descargar carga **'c'** del avión **'p'** al aeropuerto **'a'**.

Este problema puede tener múltiples soluciones válidas.
Una de esas soluciones se muestra a continuación.

In [93]:
solution = [expr("Load(C1 , P1, SFO)"),
            expr("Fly(P1, SFO, JFK)"),
            expr("Unload(C1, P1, JFK)"),
            expr("Load(C2, P2, JFK)"),
            expr("Fly(P2, JFK, SFO)"),
            expr("Unload (C2, P2, SFO)")] 

for action in solution:
    airCargo.act(action)

Como `air Cargo` ha dado todos los pasos necesarios para lograr el objetivo, ahora podemos comprobar si ha logrado su objetivo:

In [94]:
print(airCargo.goal_test())

True


Ahora ha logrado su objetivo.

## El problema de la llanta de repuesto

Consideremos el problema de cambiar la llanta pinchada de un automóvil.
El objetivo es montar una rueda de repuesto en el eje del coche, dado que tenemos una rueda pinchada en el eje y una rueda de repuesto en el maletero.

In [95]:
psource(spare_tire)

**En(obj, loc):** el objeto **'obj'** está en la ubicación **'loc'**.

**~En(obj, loc):** el objeto **'obj'** _no_ está en la ubicación **'loc'**.

**Tire(t):** Declara un neumático de tipo **'t'**.

Definamos ahora un objeto del problema `spare_tire`:

In [96]:
spareTire = spare_tire()

Antes de realizar cualquier acción, comprobaremos si `spare_tire` ha alcanzado su objetivo:

In [97]:
print(spareTire.goal_test())

False


Como podemos ver, no ha completado el objetivo.
Ahora definimos una posible solución que nos puede ayudar a alcanzar el objetivo de montar una rueda de repuesto en el eje del coche.
Luego se llevan a cabo las acciones sobre el problema de planificación de la “llanta de repuesto”.

Las acciones que tenemos a nuestra disposición son las siguientes: Quitar, Poner

**Remove(obj, loc):** Retire el neumático **'obj'** de la ubicación **'loc'**.

**PutOn(t, Eje):** Coloque el neumático **'t'** en el eje.

**LeaveOvernight():** Vivimos en un vecindario particularmente malo y todos los neumáticos, pinchados o no, nos los roban si los dejamos durante la noche.



In [98]:
solution = [expr("Remove(Flat, Axle)"),
            expr("Remove(Spare, Trunk)"),
            expr("PutOn(Spare, Axle)")]

for action in solution:
    spareTire.act(action)

In [99]:
print(spareTire.goal_test())

True


Esta es una solución válida.
<br>
Otra posible solución es

In [100]:
spareTire = spare_tire()

solution = [expr('Remove(Spare, Trunk)'),
            expr('Remove(Flat, Axle)'),
            expr('PutOn(Spare, Axle)')]

for action in solution:
    spareTire.act(action)

In [101]:
print(spareTire.goal_test())

True


Observe que ambas soluciones funcionan, lo que significa que el problema se puede resolver independientemente del orden en que se realicen las acciones "Eliminar", siempre y cuando ambas acciones "Eliminar" se realicen antes de la acción "PutOn".

Hemos montado con éxito una rueda de repuesto en el eje.

## Problema de la torre de tres bloques

El dominio de este problema consiste en un conjunto de bloques en forma de cubo colocados sobre una mesa.
Los bloques se pueden apilar, pero sólo un bloque puede caber directamente encima de otro.
Un brazo robótico puede coger un bloque y moverlo a otra posición, ya sea sobre la mesa o encima de otro bloque.
El brazo sólo puede levantar un bloque a la vez, por lo que no puede levantar un bloque que tenga otro encima.
El objetivo siempre será construir una o más pilas de bloques.
En nuestro caso, consideramos sólo tres bloques.
La configuración particular que usaremos se llama anomalía de Sussman en honor al profesor Gerry Sussman.

Echemos un vistazo a la definición de `tres_block_tower()` en el módulo.

In [102]:
psource(three_block_tower)

**On(b, x):** El bloque **'b'** está en **'x'**. **'x'** puede ser una mesa o un bloque.

**~On(b, x):** El bloque **'b'** _no_ está en **'x'**. **'x'** puede ser una mesa o un bloque.

**Bloque(b):** Declara **'b'** como un bloque.

**Borrar(x):** Para indicar que no hay nada en **'x'** y que se puede mover libremente.

**~Clear(x):** Para indicar que hay algo en **'x'** y no se puede mover.

Definamos ahora un objeto del problema `tres_bloques_torre`:

In [103]:
threeBlockTower = three_block_tower()

Antes de realizar cualquier acción, comprobaremos si `tresBlockTower` ha alcanzado su objetivo:

In [104]:
print(threeBlockTower.goal_test())

False


Como podemos ver, no ha completado el objetivo.
Ahora definimos una secuencia de acciones que pueden apilar tres bloques en el orden requerido.
Luego, las acciones se llevan a cabo en el problema de planificación "tres bloques de torres".

Las acciones que tenemos a nuestra disposición son las siguientes: MoveToTable, Move

**MoveToTable(b, x): ** Mover el cuadro **'b'** apilado en **'x'** a la tabla, dado que el cuadro **'b'** está despejado.

**Mover(b, x, y): ** Mover el cuadro **'b'** apilado en **'x'** a la parte superior de **'y'**, dado que ambos **'b '** y **'y'** son claros.


In [105]:
solution = [expr("MoveToTable(C, A)"),
            expr("Move(B, Table, C)"),
            expr("Move(A, Table, B)")]

for action in solution:
    threeBlockTower.act(action)

Como `tres_bloque_torre` ha tomado todos los pasos necesarios para lograr el objetivo, ahora podemos verificar si lo ha logrado.

In [106]:
print(threeBlockTower.goal_test())

True


Ahora ha logrado con éxito su objetivo, es decir, construir una pila de tres bloques en el orden especificado.

El problema de `tres_bloques_tower` también se puede definir en términos más simples usando solo dos acciones `ToTable(x, y)` y `FromTable(x, y)`.
Sin embargo, el problema subyacente sigue siendo el mismo: apilar tres bloques en una determinada configuración dado un estado inicial particular.
Echemos un vistazo a la definición alternativa.

In [107]:
psource(simple_blocks_world)

**On(x, y):** El bloque **'x'** está en **'y'**. Tanto **'x'** como **'y'** tienen que ser bloques.

**~On(x, y):** El bloque **'x'** _no_ está en **'y'**. Tanto **'x'** como **'y'** tienen que ser bloques.

**OnTable(x):** El bloque **'x'** está sobre la mesa.

**~OnTable(x):** El bloque **'x'** _no_ está en la mesa.

**Borrar(x):** Para indicar que no hay nada en **'x'** y que se puede mover libremente.

**~Clear(x):** Para indicar que hay algo en **'x'** y no se puede mover.

Ahora definamos un problema `simple_blocks_world`.

In [108]:
simpleBlocksWorld = simple_blocks_world()

Antes de realizar cualquier acción, veremos si `simple_bw` ha alcanzado su objetivo.

In [109]:
simpleBlocksWorld.goal_test()

False

Como podemos ver, no ha completado el objetivo.
Ahora definimos una secuencia de acciones que pueden apilar tres bloques en el orden requerido.
Luego, las acciones se llevan a cabo en el problema de planificación `simple_bw`.

Las acciones que tenemos a nuestra disposición son las siguientes: MoveToTable, Move

**ToTable(x, y): ** Mover el cuadro **'x'** apilado en **'y'** a la tabla, dado que el cuadro **'y'** está despejado.

**FromTable(x, y): ** Mover el cuadro **'x'** desde donde esté, a la parte superior de **'y'**, dado que tanto **'x'** como ** 'ustedes' ** están claros.


In [110]:
solution = [expr('ToTable(A, B)'),
            expr('FromTable(B, A)'),
            expr('FromTable(C, B)')]

for action in solution:
    simpleBlocksWorld.act(action)

Como `tres_bloque_torre` ha tomado todos los pasos necesarios para lograr el objetivo, ahora podemos verificar si lo ha logrado.

In [111]:
print(simpleBlocksWorld.goal_test())

True


Ahora ha logrado con éxito su objetivo, es decir, construir una pila de tres bloques en el orden especificado.

## Problema de compras

Este problema requiere que adquiramos un cartón de leche, un plátano y un taladro.
Inicialmente, partimos de casa y sabemos que en el supermercado se consigue leche y plátanos y en la ferretería se venden taladros.
Echemos un vistazo a la definición de "shopping_problem" en el módulo.

In [112]:
psource(shopping_problem)

**En(x):** Indica que actualmente estamos en **'x'** donde **'x'** puede ser Hogar, SM (supermercado) o HW (Ferretería).

**~At(x):** Indica que actualmente _no_ estamos en **'x'**.

**Venta(s, x):** Indica que el artículo **'x'** se puede comprar en la tienda **'s'**.

**Tener(x):** Indica que poseemos el artículo **'x'**.

In [113]:
shoppingProblem = shopping_problem()

Primero verifiquemos si se alcanza o no el estado objetivo Tener (Leche), Tener (Banana), Tener (Taladro).

In [114]:
print(shoppingProblem.goal_test())

False


Veamos las posibles acciones.

**Comprar(x, tienda):** Compra un artículo **'x'** en una **'tienda'** dado que la **'tienda'** vende **'x'**.

**Ir(x, y):** Ir al destino **'y'** comenzando desde el origen **'x'**.

Ahora definimos una solución válida que nos ayudará a alcanzar la meta.
La secuencia de acciones se llevará a cabo en el PlanningProblem `shoppingProblem`.

In [115]:
solution = [expr('Go(Home, SM)'),
            expr('Buy(Milk, SM)'),
            expr('Buy(Banana, SM)'),
            expr('Go(SM, HW)'),
            expr('Buy(Drill, HW)')]

for action in solution:
    shoppingProblem.act(action)

Hemos tomado las medidas necesarias para adquirir todo lo que necesitamos.
Veamos si hemos alcanzado nuestro objetivo.

In [116]:
shoppingProblem.goal_test()

True

Ahora ha logrado con éxito el objetivo.

## Calcetines y zapatos

Este es un simple problema de ponerse un par de calcetines y zapatos.
El problema se define en el módulo como se indica a continuación.

In [117]:
psource(socks_and_shoes)

**LeftSockOn:** Indica que ya nos hemos puesto el calcetín izquierdo.

**RightSockOn:** Indica que ya nos hemos puesto el calcetín adecuado.

**LeftShoeOn:** Indica que ya nos hemos puesto el zapato izquierdo.

**RightShoeOn:** Indica que ya nos hemos puesto el zapato adecuado.


In [118]:
socksShoes = socks_and_shoes()

Primero verifiquemos si se alcanza el estado objetivo o no.

In [119]:
socksShoes.goal_test()

False

Como no se alcanza el estado objetivo, definiremos una secuencia de acciones que podrían ayudarnos a alcanzar el objetivo.
Estas acciones luego se aplicarán sobre el problema de planificación "socksShoes" para verificar si se alcanza el estado objetivo.

In [120]:
solution = [expr('RightSock'),
            expr('RightShoe'),
            expr('LeftSock'),
            expr('LeftShoe')]

In [121]:
for action in solution:
    socksShoes.act(action)
    
socksShoes.goal_test()

True

Hemos alcanzado nuestra meta.

## Problema de pastel

Este problema requiere que lleguemos al estado de tener un pastel y haber comido un pastel simultáneamente, dado un solo pastel.
Primero echemos un vistazo a la definición del problema `have_cake_and_eat_cake_too` en el módulo.

In [122]:
psource(have_cake_and_eat_cake_too)

Dado que este problema no involucra variables, los estados pueden considerarse similares a los símbolos en lógica proposicional.

**Tener(Cake):** Declara que tenemos un **'Cake'**.

**~Have(Cake):** Declara que no tenemos un **'Cake'**.

In [123]:
cakeProblem = have_cake_and_eat_cake_too()

Primero, comprobemos si se alcanzan o no los estados objetivo 'Comer (pastel)' y 'Comer (pastel)'.

In [124]:
print(cakeProblem.goal_test())

False


Veamos las posibles acciones.

**Hornear(x):** Para hornear **' x '**.

**Comer(x):** Comer **' x '**.

Ahora definimos una solución válida que puede ayudarnos a alcanzar el objetivo.
La secuencia de acciones se aplicará entonces sobre el problema de planificación `cakeProblem`.

In [125]:
solution = [expr("Eat(Cake)"),
            expr("Bake(Cake)")]

for action in solution:
    cakeProblem.act(action)

Ahora hemos realizado acciones para hornear el pastel y comerlo. Comprobemos si hemos alcanzado la meta.

In [126]:
print(cakeProblem.goal_test())

True


Ahora ha logrado con éxito su objetivo: tener y comerse el pastel.

Cabría preguntarse si el orden de las acciones importa en este problema.
Veámoslo por nosotros mismos.

In [128]:
cakeProblem = have_cake_and_eat_cake_too()

solution = [expr('Bake(Cake)'),
            expr('Eat(Cake)')]

for action in solution:
    cakeProblem.act(action)

Exception: Action 'Bake(Cake)' pre-conditions not satisfied

Plantea una excepción.
De hecho, según el problema, no podemos hornear un pastel si ya tenemos uno.
En términos de planificación, '~Tener(Cake)' es una condición previa para la acción 'Hornear(Cake)'.
Por tanto, esta solución no es válida.

## PLANIFICACIÓN EN EL MUNDO REAL
---
## PROBLEMA
La clase `Problem` es un contenedor para `PlanningProblem` con algunas funciones y estructuras de datos adicionales para manejar problemas de planificación del mundo real que implican limitaciones de tiempo y recursos.
La clase `Problem` incluye todo lo que incluye la clase `PlanningProblem`.
Además, también incluye los siguientes atributos esenciales para definir un problema de planificación del mundo real:
- una lista de "trabajos" por realizar
- un diccionario de "recursos"

También sobrecarga el método `act` para llamar al método `do_action` de la clase `HLA`,
y también incluye un nuevo método "refinamientos" que encuentra refinamientos o acciones primitivas para acciones de alto nivel.
<br>
`hierarchical_search` y `angelic_search` también están integrados en la clase `Problem` para resolver este tipo de problemas de planificación.

In [129]:
psource(Problem)

##HLA
Para poder modelar adecuadamente un problema de planificación del mundo real, es esencial poder representar una _acción de alto nivel (HLA)_ que pueda reducirse jerárquicamente a acciones primitivas.

In [130]:
psource(HLA)

Además de las condiciones previas y los efectos, un objeto de la clase "HLA" también almacena:
- la "duración" del HLA
- la cantidad de consumo de recursos _consumibles_
- la cantidad de recursos _reutilizables_ utilizados
- un bool "completado" que indica si el "HLA" se ha completado

La clase también tiene algunos métodos auxiliares útiles:
- `do_action`: comprueba si los recursos consumibles y reutilizables necesarios están disponibles y, de ser así, ejecuta la acción.
- `has_consumable_resource`: comprueba si existe suficiente cantidad del recurso consumible requerido.
- `has_usable_resource`: comprueba si los recursos reutilizables están disponibles y no están ya comprometidos.
- `inorder`: asegura que todos los trabajos que debían ejecutarse antes del actual se han ejecutado con éxito.

## PROBLEMAS DE PLANIFICACIÓN
---
## Problema del taller de trabajo
Este es un problema sencillo que implica el montaje de dos automóviles simultáneamente.
El problema consta de dos trabajos, cada uno del formato [`AddEngine`, `AddWheels`, `Inspect`] que se realizará en dos automóviles con diferentes requisitos y disponibilidad de recursos.
<br>
Veamos cómo se ha definido `job_shop_problem` en el módulo.

In [138]:
psource(job_shop_problem)

Los estados de este problema son:
<br>
<br>
**Has(x, y)**: Coche **'x'** _has_ **'y'** donde **'y'** puede ser un Motor o una Rueda.

**~Has(x, y)**: El automóvil **'x'** no tiene _ **'y'** donde **'y'** puede ser un motor o una rueda.

**Inspeccionado(c)**: El coche **'c'** ha sido _inspeccionado_.

**~Inspeccionado(c)**: El automóvil **'c'** _no_ ha sido inspeccionado.

En el estado inicial, `C1` y `C2` son coches que no tienen motor ni ruedas y no han sido inspeccionados.
`E1` y `E2` son motores.
`W1` y `W2` son ruedas.
<br>
Nuestro objetivo es tener motores y ruedas en ambos autos y hacerlos inspeccionar. Discutiremos cómo lograrlo.
<br>
Definamos un objeto del `job_shop_problem`.

In [139]:
jobShopProblem = job_shop_problem()

Antes de realizar cualquier acción, comprobaremos si `jobShopProblem` ha alcanzado su objetivo.

In [140]:
print(jobShopProblem.goal_test())

False


Ahora definimos una posible solución que nos puede ayudar a alcanzar el objetivo.
Luego, las acciones se llevan a cabo en el objeto `jobShopProblem`.

Disponemos de las siguientes acciones:

**AddEngine1**: Agrega un motor al auto C1. Tarda 30 minutos en completarse y utiliza un polipasto de motor.

**AddEngine2**: Agrega un motor al auto C2. Tarda 60 minutos en completarse y utiliza un polipasto de motor.

**AddWheels1**: Agrega ruedas al auto C1. Tarda 30 minutos en completarse. Utiliza una estación de ruedas y consume 20 tuercas.

**AddWheels2**: Agrega ruedas al auto C2. Tarda 15 minutos en completarse. Utiliza una estación de ruedas y también consume 20 tuercas.

**Inspeccionar1**: Hace que se inspeccione el automóvil C1. Requiere 10 minutos de inspección por parte de un inspector.

**Inspeccionar2**: Hace que se inspeccione el automóvil C2. Requiere 10 minutos de inspección por parte de un inspector.

In [141]:
solution = [jobShopProblem.jobs[1][0],
            jobShopProblem.jobs[1][1],
            jobShopProblem.jobs[1][2],
            jobShopProblem.jobs[0][0],
            jobShopProblem.jobs[0][1],
            jobShopProblem.jobs[0][2]]

for action in solution:
    jobShopProblem.act(action)

In [142]:
print(jobShopProblem.goal_test())

True


Esta es una solución válida y una de las muchas formas correctas de resolver este problema.

## Doble problema de tenis
Este problema es un caso simple de un problema de planificación multiactor, donde dos agentes actúan a la vez y pueden cambiar simultáneamente el estado actual del problema.
Un plan correcto es aquel que, si lo ejecutan los actores, logra el objetivo.
En el verdadero entorno multiagente, por supuesto, es posible que los agentes no acuerden ejecutar ningún plan en particular, pero al menos sabrán qué planes _funcionarían_ si _aceptaran_ ejecutarlos.
<br>
En el problema del tenis doble, dos actores A y B juegan juntos y pueden estar en una de cuatro ubicaciones: `LeftBaseLine`, `RightBaseLine`, `LeftNet` y `RightNet`.
La pelota sólo puede ser devuelta si un jugador está en el lugar correcto.
Cada acción debe incluir al actor como argumento.
<br>
Primero veamos la definición de "problema_doble_tenis" en el módulo.

In [172]:
psource(double_tennis_problem)

Los estados de este problema son:

**Acercándose(Ball, loc)**: La `Ball` se acerca a la ubicación `loc`.

**Devuelta (pelota)**: Uno de los actores golpeó con éxito la pelota que se acercaba desde el lugar correcto, lo que provocó que regresara al otro lado.

**En(actor, loc)**: `actor` está en la ubicación `loc`.

**~En(actor, loc)**: `actor` _no_ está en la ubicación `loc`.

Ahora definamos un objeto de `double_tennis_problem`.


In [173]:
doubleTennisProblem = double_tennis_problem()

Antes de realizar cualquier acción, comprobaremos si `doubleTennisProblem` ha alcanzado la meta.

In [174]:
print(doubleTennisProblem.goal_test())

False


Como podemos ver, el objetivo no se ha alcanzado.
Definimos ahora una posible solución que nos puede ayudar a alcanzar el objetivo de que nos devuelvan el balón.
Las acciones se realizarán entonces sobre el objeto `doubleTennisProblem`.

Las acciones que tenemos a nuestra disposición son las siguientes:

**Hit(actor, ball, loc)**: devuelve una bola que se acerca si `actor` está presente en la `loc` a la que se acerca la bola.

**Go(actor, to, loc)**: mueve un `actor` desde la ubicación `loc` a la ubicación `to`.

Sin embargo, notamos algo diferente en este problema.
lo cual es bastante diferente a cualquier otro problema que hayamos visto hasta ahora.
El estado objetivo del problema contiene una variable "a".
Esto sucede a veces en problemas de planificación multiagente.
y significa que no importa _qué_ actor está en `LeftNet` o `RightNet`, siempre que haya al menos un actor en `LeftNet` o `RightNet`.

In [175]:
solution = [expr('Go(A, RightBaseLine, LeftBaseLine)'),
            expr('Hit(A, Ball, RightBaseLine)'),
            expr('Go(A, LeftNet, RightBaseLine)')]

for action in solution:
    doubleTennisProblem.act(action)

In [178]:
doubleTennisProblem.goal_test()

False

Ahora ha alcanzado con éxito su objetivo, es decir, devolver la pelota que se acerca.